In [2]:
import pandas as pd
import hashlib
import os
# displat all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
N_PARTITIONS = 100

In [ ]:
members=pd.read_csv('/Users/xuetong/Downloads/members_v3.csv',nrow=100)
transactions=pd.read_csv('/Users/xuetong/Downloads/transactions_v2.csv',nrow=100)

In [4]:
def get_header(filename):
    with open(filename, 'r') as member:
        return next(member)
members_header = get_header('/Users/xuetong/Downloads/members_v3.csv')
members_header
transactions_header = get_header('/Users/xuetong/Downloads/transactions_v2.csv')
transactions_header
train_header= get_header('/Users/xuetong/Downloads/train_v2.csv')
train_header
log_header = get_header('/Users/xuetong/Downloads/user_logs_v2.csv')
log_header
file_headers={'members.csv':members_header,
   'transactions.csv':transactions_header,
   'train.csv':train_header,
    'logs.csv':log_header}

'msno,city,bd,gender,registered_via,registration_init_time\n'

'msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel\n'

'msno,is_churn\n'

'msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs\n'

In [13]:
base_dir = '/Users/xuetong/customer_churn/data_partition/'

if not os.path.exists(base_dir + 'p99'):
    # Create a new directory for each partition
    for i in range(N_PARTITIONS):
        os.makedirs(base_dir + f'p{i}', exist_ok=False)
    
len(os.listdir(base_dir))

100

In [6]:
base_dir = '/Users/xuetong/customer_churn/data_partition/'
def create_blank_partitions():
    """Create blank files in each partition and write the file header"""
    # For each partition create the files with headers
    for i in range(N_PARTITIONS):
        directory = base_dir + f'p{i}/'
        # Create five files
        for file in ['transactions.csv', 'train.csv', 'members.csv', 'logs.csv']:
            # Write file header as first line
            with open(directory + file, 'w') as f:
                f.write(file_headers[file])
create_blank_partitions()

In [7]:
def id_to_hash(customer_id):
    """
    Hash customer_id to 16_bit integer
    Params:
    --------
    customer_id (str): unique id of each customer
    
    Returns:
    --------
    a 16-bit integer hash of a customer id string
    """
    return int(hashlib.md5(customer_id.encode('utf-8')).hexdigest(), 16)

In [8]:
def partition_by_hashing(df, name):
    """Partition a dataframe into N_PARTITIONS by hashing the id.
    
    Params
    --------
        df (pandas dataframe): dataframe for partition. Must have 'msno' column.
        name (str): name of dataframe. Used for saving the row data.
                                  
    Returns:
    --------
        Nothing returned. Dataframe is saved one line at a time as csv files to the N_PARTITIONS 
    """
    df['partition'] = df['msno'].apply(id_to_hash) % N_PARTITIONS
    
    for partition, grouped in df.groupby('partition'):
        grouped = grouped.drop(columns = 'partition')
        with open(base_dir + f'p{partition}/{name}.csv', 'a') as f:
            f.write('\n')
            grouped.to_csv(f, header = False, index = False)
    

In [21]:
members=pd.read_csv('/Users/xuetong/Downloads/members_v3.csv')
partition_by_hashing(members, name='members')

In [23]:
pd.read_csv(base_dir + f'p3/members.csv').head()

,msno,city,bd,gender,registered_via,registration_init_time
0,+zs+1G87PFI6/GGDHRZirqUwuqWwxHfbYXCDp6JASIA=,5,20,male,3,20141021
1,rfRIfohiDPl+45b/U18yoSh4BxyzUgfzIN37HJx6uwA=,15,33,female,3,20141026
2,XOtdjEYPuLTlLrVtm+SuVU6G4RyBsuvoag1SETzPYBE=,22,22,male,3,20141105
3,aW4IA1Wl1hB0/4id06Fn77rLUFgzjGbmx9mUUigN/74=,1,0,NaN,11,20161224
4,hnQkTaqdbYdNySpKuCpHAFKo2eLC0v5h992Fs4lKozs=,1,21,female,4,20161229


In [24]:
transactions = pd.read_csv('/Users/xuetong/Downloads/transactions_v2.csv')
partition_by_hashing(transactions, name = 'transactions')

In [26]:
pd.read_csv(base_dir + f'p3/transactions.csv').head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
1,/3T/9RetFt4UGlip/yfz8W8IR1PsleTWMMqfDRniYtA=,39,30,149,149,1,20170331,20170511,0
2,0e0apAPtBl6gtqeMROaBHa7syvdcYA98cjjWla6yDLM=,41,30,99,99,1,20170321,20170421,0
3,1V+osC/XWRFOU7Ozwk73Cs37HfVFEwyC9+x4hgqfNzg=,41,30,149,149,1,20170321,20170421,0
4,2bP8ThbrGbvx5JN81VjxeFkqawTD53RkLH7s3MwtXyw=,32,410,1788,1788,0,20170308,20180425,0


In [25]:
user_log=pd.read_csv('/Users/xuetong/Downloads/user_logs_v2.csv')
partition_by_hashing(user_log, name = 'user_log')

In [10]:
train=pd.read_csv('/Users/xuetong/Downloads/train_v2.csv')
train.head()
partition_by_hashing(train,name='train')

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1
